In [8]:
import openpyxl as px
import plotly as plt
import plotly.graph_objects as go

In [170]:
# course data is available from Fall of 2000 through Spring of 2021

semesters = ["Fall 2000", "Spring 2001", "Fall 2001", "Spring 2002", "Fall 2002", "Spring 2003", "Fall 2003",
             "Spring 2004", "Fall 2004", "Spring 2005", "Fall 2005", "Spring 2006", "Fall 2006", "Spring 2007",
             "Fall 2007", "Spring 2008", "Fall 2008", "Spring 2009", "Fall 2009", "Spring 2010", "Fall 2010", 
             "Spring 2011", "Fall 2011", "Spring 2012", "Fall 2012", "Spring 2013", "Fall 2013", "Spring 2014",
             "Fall 2014", "Spring 2015", "Fall 2015", "Spring 2016", "Fall 2016", "Spring 2017", "Fall 2017",
             "Spring 2018", "Fall 2018", "Spring 2019", "Fall 2019", "Spring 2020", "Fall 2020", "Spring 2021"]

# Examining a single department over time

The percentFilled() function takes a single semester and a department acronym (e.g. "CS" for the Computer Science department). The function accesses the xlsx file that corresponds to the inputted semester, and calculates the total enrollment for all courses and the average percent filled for the entire department. Note that the total enrollment can represent the same student twice if the student enrolled in two or more courses in that department within the same semester. This information is returned as a tuple in the form (total enrollment, average percent filled). 

In [163]:
def percentFilled(semester, department):
    fileName = "WellesleyCourses/" + semester + ".xlsx"
    
    wb = px.load_workbook(filename = fileName)
    ws = wb.active
    numRows = ws.max_row
    coursesFormatted = []
    
    numCourses = 0
    totalEnrollment = 0
    totalPercent = 0
    for i in range(2, numRows + 1):
        course = ws.cell(row = i, column = 2).value
        dpt = course.split()[0]
        if dpt == department:
            enrollment = ws.cell(row = i, column = 4).value
            seats = ws.cell(row = i, column = 5).value
            numerator = int(seats.split()[0])
            denominator = int(seats.split()[2])
            fraction = numerator / denominator
            percent = 1 - fraction
                
            numCourses += 1
            totalEnrollment += int(enrollment)
            totalPercent += percent
            
    if numCourses == 0:
        return (0, 0)
    else:               
        return (totalEnrollment, round(totalPercent / numCourses, 2))

To use the function, call percentFilled() with two parameters: first, a semester from the 'semesters' list in cell 2, and a department acronym. Both must be inputted as strings. 

In [164]:
percentFilled("Spring 2015", "WRIT")

(326, 0.93)

The courseHistory() function takes a single department acronym as a string and produces a graph that shows how full the classes in that department were from Fall 2000 through Spring 2021. 

In [168]:
def courseHistory(department):
    percentages = []
    enrollment = []
    for sem in semesters:
        en, p = percentFilled(sem, department)
        percentages.append(p)
        enrollment.append( str(en) + " total enrollment(s)")
        
    fig = go.Figure(data=[go.Bar(x=semesters, y=percentages, hovertext=enrollment)])
    fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)', 
                          marker_line_width=1.5, opacity=0.6)
    fig.update_layout(title_text=department, uniformtext_mode='show', uniformtext_minsize=2)
    fig.show()


To use the courseHistory() function, input a department acronym as a string. Hover over each bar to see the total enrollment during the corresponding semester. 

In [169]:
courseHistory("CS")

# Comparing multiple departments

The final function in this notebook produces a graph that compares how full the classes in every department were during a specific semester. The function accesses an xlsx file that corresponds to the inputted semester, and stores information about each department and the courses it offered. It stores the information in a series of dictionaries that aggregate and format the data to be used for the graph. 

In [47]:
def makeGraph(semester):
    
    fileName = "WellesleyCourses/" + semester + ".xlsx"
    
    wb = px.load_workbook(filename = fileName)
    ws = wb.active
    numRows = ws.max_row
    coursesFormatted = []
    
    for i in range(2, numRows + 1):
        course = ws.cell(row = i, column = 2).value
        department = course.split()[0]
        num = course.split()[1]
        enrollment = ws.cell(row = i, column = 4).value
        seats = ws.cell(row = i, column = 5).value
        numerator = int(seats.split()[0])
        denominator = int(seats.split()[2])
        fraction = round(numerator / denominator, 2)
        percentFilled = round(1 - fraction, 2)
        coursesFormatted.append((department, num, enrollment, fraction, percentFilled))
        
    fileDict = {}
    avgsDict = {}
    
    for course in coursesFormatted:
        department = course[0]
        enrollment = int(course[2])
        percentFilled = course[4]
    
        if department not in fileDict.keys():
            fileDict[department] = (1, enrollment, percentFilled)

        else:
            (N, E, P) = fileDict[department]
            N += 1
            E += enrollment
            P += percentFilled
            fileDict[department] = (N, E, P)
            
    for item in fileDict.items():
        dept = item[0]
        num = item[1][0]
        enrollment = item[1][1]
        totalPercent = item[1][2]
        avgsDict[dept] = (num, enrollment, round(totalPercent / num, 2))
        
    allDepts = []
    allNumCourses = []
    allNumEnrollments = []
    allAvgFilled = []

    for item in avgsDict.items():
        dept = item[0]
        allDepts.append(dept)
        numCourses = item[1][0]
        allNumCourses.append(str(numCourses) + " total courses")
        enrollment = item[1][1]
        allNumEnrollments.append(str(enrollment) + " enrollment(s)")
        avgFilled = item[1][2]
        allAvgFilled.append(avgFilled)
        
    fig = go.Figure(data=[go.Bar(x=allDepts, y=allAvgFilled, hovertext=allNumEnrollments)])

    fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)',
                      marker_line_width=1.5, opacity=0.6)
    fig.update_layout(title_text=semester, xaxis={'categoryorder':'total ascending'}, 
                      uniformtext_mode='show', uniformtext_minsize=2)
    fig.show()


To create a graph, choose a semester between Fall 2000 and Spring 2021. Call the function makeGraph() on that semester (as a string) to produce the graph that corresponds to that semester. Hover over each bar to see the total enrollment in that department during the given semester. 

In [107]:
makeGraph("Spring 2015")